## Import and Install    

In [ ]:
!pip install bs4
    #!pip install requests

    from bs4 import BeautifulSoup # this module helps in web scrapping.
    import requests  # this module helps us to download a web page

    import pandas as pd
    import requests
    import numpy as np

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
!pip install folium
import folium 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 4.1 MB/s  eta 0:00:01


In [4]:
from sklearn.cluster import KMeans

In [5]:
!pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 8.2 MB/s  eta 0:00:01


In [6]:
!pip install geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [7]:
import geocoder

In [8]:
html_data = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=906439794').text 

In [9]:
bs=BeautifulSoup(html_data, 'html5lib')

In [10]:
tables = bs.find_all('table')

In [11]:
for index,table in enumerate(tables):
    if ("M1A" in str(table)):
        table_index = index
print(table_index)

0


In [12]:
toronto_data = pd.DataFrame(columns=["Postal_Code", "Borough", "Neighbourhood"])


In [13]:
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    for i in range(3):
        if (col != []):
            Postal_Code = col[0].text
            Borough =col[1].text
            Neighbourhood =col[2].text
            toronto_data = toronto_data.append({"Postal_Code":Postal_Code, "Borough":Borough, "Neighbourhood":Neighbourhood}, ignore_index=True)

## Clean dataframe

In [14]:
toronto_data.replace('\n','', regex=True, inplace = True)

In [15]:
df2=toronto_data[(toronto_data.Borough !="Not assigned")]
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])
df3=df2.groupby(['Postal_Code','Borough'], sort=False).agg(', '.join)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [16]:
df3.reset_index(inplace=True)

In [17]:
df3.head(10)

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Rege..."
3,M6A,North York,"Lawrence Heights, Lawrence Heights, Lawrence H..."
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park"
5,M9A,Etobicoke,"Islington Avenue, Islington Avenue, Islington ..."
6,M1B,Scarborough,"Rouge, Rouge, Rouge, Malvern, Malvern, Malvern"
7,M3B,North York,"Don Mills North, Don Mills North, Don Mills North"
8,M4B,East York,"Woodbine Gardens, Woodbine Gardens, Woodbine G..."
9,M5B,Downtown Toronto,"Ryerson, Ryerson, Ryerson, Garden District, Ga..."


In [18]:
df4=df3

## Get Longitude and Latitude from geocoders


In [19]:
lat_lng_coords = ["Postal_code","Latitude","Longitude"]

In [20]:
from geopy.geocoders import Nominatim

In [21]:
def eval_results(x):
    try:
        return [x.latitude, x.longitude]
    except:
        return [None, None]

In [22]:
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(103):
  location = geolocator.geocode('{}, Toronto, Ontario'.format(df3.Postal_Code[i]))
  location1 = eval_results(location)
  latitude = location1[0]
  longitude = location1[1]
  lat_lng_coords.append([df3.Postal_Code[i],latitude,longitude])
 # print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))
 

In [23]:
lat_lng = pd.DataFrame(lat_lng_coords[3:106])
lat_lng


,0,1,2
0,M3A,43.652384,-79.383568
1,M4A,NaN,NaN
2,M5A,NaN,NaN
3,M6A,NaN,NaN
4,M7A,43.652384,-79.383568
...,...,...,...
98,M8X,NaN,NaN
99,M4Y,NaN,NaN
100,M7Y,NaN,NaN
101,M8Y,NaN,NaN


In [24]:
lat_lng.columns = ['Postal_Code','Longitude','Latitude']


In [25]:
lat_lng

,Postal_Code,Longitude,Latitude
0,M3A,43.652384,-79.383568
1,M4A,NaN,NaN
2,M5A,NaN,NaN
3,M6A,NaN,NaN
4,M7A,43.652384,-79.383568
...,...,...,...
98,M8X,NaN,NaN
99,M4Y,NaN,NaN
100,M7Y,NaN,NaN
101,M8Y,NaN,NaN


In [26]:
df4 = pd.merge(df3,lat_lng,on='Postal_Code')
df4.head()

,Postal_Code,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods",43.652384,-79.383568
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V...",NaN,NaN
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Rege...",NaN,NaN
3,M6A,North York,"Lawrence Heights, Lawrence Heights, Lawrence H...",NaN,NaN
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park",43.652384,-79.383568


In [27]:
df4.shape

(103, 5)

## Get Longitude and Latitude from excel

In [28]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
lat_lon.rename(columns={'Postal Code':'Postal_Code'},inplace=True)


In [32]:
df6 = pd.merge(df3,lat_lon,on='Postal_Code')
df6.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods",43.753259,-79.329656
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Rege...",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Heights, Lawrence H...",43.718518,-79.464763
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park",43.662301,-79.389494


## Toronto as Borough

In [33]:
df7 = df6[df6['Borough'].str.contains('Toronto',regex=False)]

In [34]:
df7

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Rege...",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Ryerson, Ryerson, Garden District, Ga...",43.657162,-79.378937
15,M5C,Downtown Toronto,"St. James Town, St. James Town, St. James Town",43.651494,-79.375418
19,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031
20,M5E,Downtown Toronto,"Berczy Park, Berczy Park, Berczy Park",43.644771,-79.373306
24,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street, Centra...",43.657952,-79.387383
25,M6G,Downtown Toronto,"Christie, Christie, Christie",43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, Adelaide, Adelaide, King, King, King...",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dovercourt Village, Doverc...",43.669005,-79.442259
36,M5J,Downtown Toronto,"Harbourfront East, Harbourfront East, Harbourf...",43.640816,-79.381752


## Map

In [35]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df7['Latitude'],df7['Longitude'],df7['Borough'],df7['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

## Cluster

In [36]:
k=5
toronto_clustering = df7.drop(['Postal_Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df7.insert(0, 'Cluster Labels', kmeans.labels_)

In [37]:
df7

,Cluster Labels,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
2,2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Rege...",43.654260,-79.360636
9,2,M5B,Downtown Toronto,"Ryerson, Ryerson, Ryerson, Garden District, Ga...",43.657162,-79.378937
15,2,M5C,Downtown Toronto,"St. James Town, St. James Town, St. James Town",43.651494,-79.375418
19,0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031
20,2,M5E,Downtown Toronto,"Berczy Park, Berczy Park, Berczy Park",43.644771,-79.373306
24,2,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street, Centra...",43.657952,-79.387383
25,1,M6G,Downtown Toronto,"Christie, Christie, Christie",43.669542,-79.422564
30,2,M5H,Downtown Toronto,"Adelaide, Adelaide, Adelaide, King, King, King...",43.650571,-79.384568
31,4,M6H,West Toronto,"Dovercourt Village, Dovercourt Village, Doverc...",43.669005,-79.442259
36,2,M5J,Downtown Toronto,"Harbourfront East, Harbourfront East, Harbourf...",43.640816,-79.381752


## Cluster on Map

In [39]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df7['Latitude'], df7['Longitude'], df7['Neighbourhood'], df7['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters